## Ticker Data Filitering 

#### Overview
This file filters a tickers dataset, `Raw_NASDAQ_Listings.csv`, that includes all the symbols of companies currently being traded on NASDAQ. The `Raw_NASDAQ_Listings.csv` includes the folllowing columns: Symbol, Name, Last_Sale, Net_Change, %_Change, Market_Cap, Country, IPO_Year, Volume. This file filters out all rows including company subsidaries, all companies which have IPOed in the last two years, and companies that are not registered to the United States and saves it in a new csv: `Filtered_NASDAQ_Listings.csv`. The `Filtered_NASDAQ_Listings.csv` is used as the input for the `scrape_aggregate_data()`. The `scrape_aggregate_data()` function iterates through the Symbols column in `Filtered_NASDAQ_Listings.csv` and creates a new directory with a Symbol name and the starts scraping the data from Polygon.io 3 months at a time and then places the collection of 2 years worth of stock data in 3 month intervals into the Symbol's respective directory. However sice the Polygon.io API allows for only 5 API calls per minute with each API call limited to 50,000 trades every call, we can only scrape 3 months data every 20 seconds. Sicne our `Filtered_NASDAQ_Listings.csv` has a total of 2640 Symbols with each symbol requiring a total of 8 csv files including 3 months of data each we will need to scrape data for 2 minutes for each Symbol (upper bound). Since we collecting data for 2640 Symbols our total data scraping time will be 88 hours (upper bound). In order to make this more manageable we will be splitting the `Filtered_NASDAQ_Listings.csv` in half with each partner being responsible for collecting the data for half of the Symbols (1320 Symbols each). The `Filtered_NASDAQ_Listings.csv` was filtered to only include the Symbols column and the split into `Symbols_First_half.csv` and `Symbols_Second_half.csv`.

In [2]:
import pandas as pd

df = pd.read_csv('Raw_NASDAQ_Listings.csv')

print(df)

     Symbol                                               Name Last_Sale  \
0         A             Agilent Technologies Inc. Common Stock  $151.28    
1        AA                    Alcoa Corporation Common Stock    $39.83    
2      AACG   ATA Creativity Global American Depositary Shares    $0.92    
3      AACI            Armada Acquisition Corp. I Common Stock   $11.40    
4     AACIU                    Armada Acquisition Corp. I Unit   $11.11    
...     ...                                                ...       ...   
7158   ZVRA               Zevra Therapeutics Inc. Common Stock    $4.80    
7159   ZVSA             ZyVersa Therapeutics Inc. Common Stock    $5.29    
7160    ZWS  Zurn Elkay Water Solutions Corporation Common ...   $32.56    
7161   ZYME                        Zymeworks Inc. Common Stock    $9.00    
7162   ZYXI                            Zynex Inc. Common Stock   $10.86    

      Net_Change % Change    Market_Cap        Country  IPO_Year   Volume  \
0         

In [3]:
# Drops all tickers that are not registered to the US
df = df[df.Country == "United States"]

print(df)

     Symbol                                               Name Last_Sale  \
0         A             Agilent Technologies Inc. Common Stock  $151.28    
1        AA                    Alcoa Corporation Common Stock    $39.83    
3      AACI            Armada Acquisition Corp. I Common Stock   $11.40    
4     AACIU                    Armada Acquisition Corp. I Unit   $11.11    
5     AACIW                 Armada Acquisition Corp. I Warrant    $0.13    
...     ...                                                ...       ...   
7158   ZVRA               Zevra Therapeutics Inc. Common Stock    $4.80    
7159   ZVSA             ZyVersa Therapeutics Inc. Common Stock    $5.29    
7160    ZWS  Zurn Elkay Water Solutions Corporation Common ...   $32.56    
7161   ZYME                        Zymeworks Inc. Common Stock    $9.00    
7162   ZYXI                            Zynex Inc. Common Stock   $10.86    

      Net_Change % Change    Market_Cap        Country  IPO_Year   Volume  \
0         

In [4]:
# Drops all tickers that IPOed in the last 2 years (non-inclusive)
df = df[df.IPO_Year < 2022]

print(df)

     Symbol                                               Name Last_Sale  \
0         A             Agilent Technologies Inc. Common Stock  $151.28    
1        AA                    Alcoa Corporation Common Stock    $39.83    
3      AACI            Armada Acquisition Corp. I Common Stock   $11.40    
4     AACIU                    Armada Acquisition Corp. I Unit   $11.11    
5     AACIW                 Armada Acquisition Corp. I Warrant    $0.13    
...     ...                                                ...       ...   
7152    ZTS                   Zoetis Inc. Class A Common Stock  $168.87    
7153   ZUMZ                           Zumiez Inc. Common Stock   $18.51    
7154    ZUO                    Zuora Inc. Class A Common Stock   $10.27    
7157   ZVIA                     Zevia PBC Class A Common Stock    $1.08    
7160    ZWS  Zurn Elkay Water Solutions Corporation Common ...   $32.56    

      Net_Change % Change    Market_Cap        Country  IPO_Year   Volume  \
0         

In [11]:
# Now the filtered data can be saved as a new csv called Filtered_NASDAQ_Listings.csv
file_name = "Filtered_NASDAQ_Listings.csv"
df.to_csv(file_name, index=True)

print(f"Filtered DataFrame saved as {file_name} in the current directory: tickers_dataset")

Filtered DataFrame saved as Filtered_NASDAQ_Listings.csv in the current directory: tickers_dataset


In [15]:
# Since Polygon.io only allows for 5 API calls per minute and each API is allowed 50,000 ticker 
# datums we unable to stream all of the minute by minute data for one ticker at a time  


df = pd.read_csv('Filtered_NASDAQ_Listings.csv')

# Filters df to only include symbol columns and index
symbols = "Symbols_df"
symbols_df = df[["Symbol"]]
print(symbols_df)
symbols_df.to_csv(symbols, index=True)
print(f"Filtered DataFrame saved as {symbols} in the current directory: tickers_dataset")

     Symbol
0         A
1        AA
2      AACI
3     AACIU
4     AACIW
...     ...
2635    ZTS
2636   ZUMZ
2637    ZUO
2638   ZVIA
2639    ZWS

[2640 rows x 1 columns]
Filtered DataFrame saved as Symbols_df in the current directory: tickers_dataset


In [9]:
# Splitting the symbols_df in half for each partner to process
midpoint = len(symbols_df) // 2
df_first_half = symbols_df.iloc[:midpoint].copy()
df_second_half = symbols_df.iloc[midpoint:].copy()

# First half of the symbols_df
print("First half of the DataFrame:")
print(df_first_half)

# Second half of the symbols_df
print("\nSecond half of the DataFrame:")
print(df_second_half)

First half of the DataFrame:
     Symbol
0         A
1        AA
2      AACI
3     AACIU
4     AACIW
...     ...
1315   JRSH
1316   JSPR
1317  JSPRW
1318    JVA
1319    JXN

[1320 rows x 1 columns]

Second half of the DataFrame:
     Symbol
1320   JYNT
1321     KA
1322    KAI
1323   KALA
1324    KAR
...     ...
2635    ZTS
2636   ZUMZ
2637    ZUO
2638   ZVIA
2639    ZWS

[1320 rows x 1 columns]


In [10]:
# Save each half of the symbols_df as a csv
# First half of the symbols_df saved as Symbols_First_Half.csv
symbols_first_half = "Symbols_First_Half.csv"
df_first_half.to_csv(symbols_first_half , index=True)
print(f"Filtered DataFrame saved as {symbols_first_half} in the current directory: tickers_dataset")

# Second half of the symbols_df saved as Symbols_Second_Half.csv
symbols_second_half = "Symbols_Second_Half.csv"
df_second_half.to_csv(symbols_second_half , index=True)
print(f"Filtered DataFrame saved as {symbols_second_half} in the current directory: tickers_dataset")

Filtered DataFrame saved as Symbols_First_Half.csv in the current directory: tickers_dataset
Filtered DataFrame saved as Symbols_Second_Half.csv in the current directory: tickers_dataset
